In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

In [3]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='file:///c:/Users/khadk/Desktop/MLOps/training/experimental_tracking/mlruns/1', creation_time=1729173508282, experiment_id='1', last_update_time=1729173508282, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
#@ code reconstruction:
def read_dataframe(filename):
    df=pd.read_parquet(filename)

    df['duration']=df.lpep_dropoff_datetime-df.lpep_pickup_datetime
    df.duration=df.duration.apply(lambda td: td.total_seconds()/60)

    df[((df.duration >= 1) & (df.duration <=60))]
    categorical=['PULocationID', 'DOLocationID']

    df[categorical]=df[categorical].astype(str)
    
    return df

In [5]:
df_train=read_dataframe('C:/Users/khadk/Desktop/MLOps/Data_set/green_tripdata_2024-01.parquet')
df_val=read_dataframe('C:/Users/khadk/Desktop/MLOps/Data_set/green_tripdata_2024-02.parquet')                  

In [6]:
df_train['PO_DO']=df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PO_DO']=df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical=['PO_DO']#'PULocationID', 'DOLocationID']
numerical=['trip_distance']

dv=DictVectorizer()

train_dicts=df_train[categorical + numerical].to_dict(orient='records')
X_train=dv.fit_transform(train_dicts)

val_dicts=df_val[categorical + numerical].to_dict(orient='records')
X_val=dv.transform(val_dicts)

In [8]:
target='duration'
y_train=df_train[target].values
y_val=df_val[target].values

In [9]:
lr=LinearRegression()
lr.fit(X_train, y_train)

y_pred=lr.predict(X_val)


mean_squared_error(y_val, y_pred, squared=False)

78.80821659744552

In [10]:
import os 

os.makedirs('Models', exist_ok=True)

with open('Models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr),f_out)

In [14]:
with mlflow.start_run():
    
    mlflow.set_tag('developer', 'king')
    mlflow.log_param('train-data-path', 'C:/Users/khadk/Desktop/MLOps/Data_set/green_tripdata_2024-01.parquet')
    mlflow.log_param('valid-data-path', 'C:/Users/khadk/Desktop/MLOps/Data_set/green_tripdata_2024-02.parquet')
    alpha=0.001 
    mlflow.log_param('alpha', alpha)
    lr=Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred=lr.predict(X_val)

    from sklearn.metrics import mean_squared_error

    rmse=mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

    mlflow.log_artifact(local_path='Models/lin_reg.bin', artifact_path='models_pickle' )

In [19]:
import  xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [20]:
train=xgb.DMatrix(X_train, label=y_train)
valid=xgb.DMatrix(X_val, label=y_val)

In [37]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster=xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )

        y_pred=booster.predict(valid)
        rmse=mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    
    return {'loss': rmse, 'status': STATUS_OK}

In [38]:

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

C:\Users\khadk\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:160: UserWarning: [20:54:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:77.41365                          
[1]	validation-rmse:77.19018                          
[2]	validation-rmse:77.05528                          
[3]	validation-rmse:76.95915                          
[4]	validation-rmse:76.90849                          
[5]	validation-rmse:76.89007                          
[6]	validation-rmse:76.88785                          
[7]	validation-rmse:76.90035                          
[8]	validation-rmse:76.92181                          
[9]	validation-rmse:76.95216                          
[10]	validation-rmse:76.98217                         
[11]	validation-rmse:77.01765                         
[12]	validation-rmse:77.06101                         
[13]	validation-rmse:77.10482                         
[14]	validation-rmse:77.13200                         
[15]	validation-rmse:77.16096                         
[16]	validation-rmse:77.20123                         
 10%|█         | 1/10 [00:08<01:20,  8.98s/trial, best loss: 77.2

C:\Users\khadk\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:160: UserWarning: [20:54:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:77.38260                                                   
[1]	validation-rmse:77.38545                                                   
[2]	validation-rmse:77.55379                                                   
[3]	validation-rmse:77.80297                                                   
[4]	validation-rmse:78.04902                                                   
[5]	validation-rmse:78.32359                                                   
[6]	validation-rmse:78.59441                                                   
[7]	validation-rmse:78.76902                                                   
[8]	validation-rmse:78.94217                                                   
[9]	validation-rmse:79.15886                                                   
[10]	validation-rmse:79.34799                                                  
 20%|██        | 2/10 [00:22<01:32, 11.59s/trial, best loss: 77.20122573862926]

...

[0]	validation-rmse:79.43500                                                   
[1]	validation-rmse:83.44550                                                   
[2]	validation-rmse:86.34542                                                   
[3]	validation-rmse:88.08416                                                   
[4]	validation-rmse:88.94902                                                   
[5]	validation-rmse:89.28686                                                   
[6]	validation-rmse:89.48519                                                   
[7]	validation-rmse:89.52215                                                   
[8]	validation-rmse:89.52407                                                   
[9]	validation-rmse:89.52808                                                   
[10]	validation-rmse:89.53601                                                  
[0]	validation-rmse:76.96571                                                   
[1]	validation-rmse:76.74047            

C:\Users\khadk\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:160: UserWarning: [20:55:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[3]	validation-rmse:76.73436                                                   
[4]	validation-rmse:76.73696                                                   
[5]	validation-rmse:76.80795                                                   
[6]	validation-rmse:76.80164                                                   
[7]	validation-rmse:76.80454                                                   
[8]	validation-rmse:76.82888                                                   
[9]	validation-rmse:76.86967                                                   
[10]	validation-rmse:76.87453                                                  
[11]	validation-rmse:76.87931                                                  
[12]	validation-rmse:76.91081                                                  
 90%|█████████ | 9/10 [01:16<00:09,  9.00s/trial, best loss: 76.91081144981564]

C:\Users\khadk\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:160: UserWarning: [20:55:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:77.54354                                                   
[1]	validation-rmse:77.38953                                                   
[2]	validation-rmse:77.27541                                                   
[3]	validation-rmse:77.19323                                                   
[4]	validation-rmse:77.14364                                                   
[5]	validation-rmse:77.11293                                                   
[6]	validation-rmse:77.10248                                                   
[7]	validation-rmse:77.10125                                                   
[8]	validation-rmse:77.11460                                                   
[9]	validation-rmse:77.12200                                                   
[10]	validation-rmse:77.13335                                                  
[11]	validation-rmse:77.15886                                                  
[12]	validation-rmse:77.19080           

In [39]:
with mlflow.start_run():
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)


    best_params={
        'learning_rate': 0.09070900080602705,
        'max_depth': 100,
        'min_child_weight': 8.576508447956027,
        'objective': 'reg:linear',
        'reg_alpha': 0.024561332497982886,    
        'reg_lambda':0.07009727399544963,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open('Models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact('Models/preprocessor.b', artifact_path='preprocessor')
    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')


C:\Users\khadk\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:160: UserWarning: [21:05:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:77.54354
[1]	validation-rmse:77.38953
[2]	validation-rmse:77.27541
[3]	validation-rmse:77.19323
[4]	validation-rmse:77.14364
[5]	validation-rmse:77.11293
[6]	validation-rmse:77.10248
[7]	validation-rmse:77.10125
[8]	validation-rmse:77.11460
[9]	validation-rmse:77.12200
[10]	validation-rmse:77.13335
[11]	validation-rmse:77.15886
[12]	validation-rmse:77.19080
[13]	validation-rmse:77.20652
[14]	validation-rmse:77.22871
[15]	validation-rmse:77.26185
[16]	validation-rmse:77.29702


C:\Users\khadk\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:160: UserWarning: [21:05:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
2024/10/18 21:06:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [40]:
logged_model = 'runs:/bae3db8d3c47435eb53e9571686f03e5/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

C:\Users\khadk\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:160: UserWarning: [21:11:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [41]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: bae3db8d3c47435eb53e9571686f03e5

In [43]:
xgboost_model=mlflow.xgboost.load_model(logged_model)
xgboost_model

C:\Users\khadk\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:160: UserWarning: [21:13:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [44]:
#@ Prediction:
y_pred=xgboost_model.predict(valid)

In [45]:
y_pred[:10]

array([21.20176  , 27.899284 ,  9.062972 , 10.35958  , 20.690088 ,
        6.5452857,  6.5452857, 10.35958  , 22.176706 , 31.00552  ],
      dtype=float32)